In [ ]:
import numpy as np

import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Data

In [ ]:
df = pd.DataFrame(data)

choose column to predict

In [ ]:
COLUMN_TO_PREDICT = 'Close'

In [ ]:
data_to_predict = df[[COLUMN_TO_PREDICT]].values

data scaling

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_to_predict)

creating training sequences

In [ ]:
from services.model_services.time_series.lstm.src.utils import create_sequences

window_size = 5  # setting the length of the prediction window
x_train, y_train = create_sequences(scaled_data, window_size)

convert to tensor

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

define Dataset

In [ ]:
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Training model

set hiperparameters

In [ ]:
# experiment hiperparameters
learning_rate = 0.001
num_epochs = 100
criterion = nn.MSELoss()

In [ ]:
# model hiperparameters
input_size = 1
hidden_size = 64
output_size = 1
num_layers = 2

define model and optimizer

In [ ]:
from services.model_services.time_series.lstm.src.lstm_model import LSTMModel

model = LSTMModel(input_size, hidden_size, output_size, num_layers)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model training

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}')

# Prediction

predict last values

In [ ]:
model.eval()
test_seq = torch.tensor(scaled_data[-window_size:], dtype=torch.float32).unsqueeze(0)
predicted_price = model(test_seq).detach().numpy()

inverse scaling

In [ ]:
predicted_price = scaler.inverse_transform(predicted_price)
print(f'predicted value: {round(predicted_price[0][0],3)}')